In [1]:
%load_ext autoreload

import os, sys
sys.path.insert(0, '../')

In [2]:
%autoreload
from extract_utils.querier import *
from extract_utils.display_utils import *

import itertools, os, psycopg2, re, sys, time, numpy as np, pandas as pd
from sklearn import metrics
from datetime import datetime
from datetime import timedelta
from collections import Counter, defaultdict

from os.path import isfile, isdir, splitext
import argparse
import pickle as cPickle
import numpy.random as npr

SQL_DIR = os.path.join('./SQL_Queries')
STATICS_QUERY_PATH = os.path.join(SQL_DIR, 'statics.sql')
LABS_QUERY_PATH = os.path.join(SQL_DIR, 'labs.sql')
VITALS_QUERY_PATH = os.path.join(SQL_DIR, 'vitals.sql')
TREATMENTS_QUERY_PATH = os.path.join(SQL_DIR, 'treatments.sql')
CODES_QUERY_PATH = os.path.join(SQL_DIR, 'codes.sql')

In [3]:
BLINDED = True

# Questions on eICU
  1. In `lab`, `labresultoffset` vs. `labresultrevisedoffset`?

### Set up Args

In [4]:
idx = pd.IndexSlice

In [5]:
args = {
    # Querier Params
    'psql_host': '/var/run/postgresql/',
    
    # Pop Restrictions
    'pop_size': 3000000,
    'min_age': 15,
    'min_duration': 24,
    'max_duration': 240,
    'min_labs_thresh': 5,
}

### Set up Querier

In [6]:
dbname = 'eicu'
schema_name = 'public'
query_args = {'dbname': dbname}
if args['psql_host'] is not None: query_args['host'] = args['psql_host']
# if args['psql_user'] is not None: query_args['user'] = args['psql_user']
# if args['psql_password'] is not None: query_args['password'] = args['psql_password']

querier = Querier(query_args=query_args, schema_name=schema_name)

### Helper Functions

In [7]:
def range_unnest(df, col, out_col_name=None, reset_index=False):
    assert len(df.index.names) == 1, "Does not support multi-index."
    if out_col_name is None: out_col_name = col

    col_flat = pd.DataFrame(
        [[i, x] for i, y in df[col].iteritems() for x in range(y+1)],
        columns=[df.index.names[0], out_col_name]
    )

    if not reset_index: col_flat = col_flat.set_index(df.index.names[0])
    return col_flat

### Extract Cohort

In [8]:
#############
# Population extraction

start = time.time()

print("Building data from scratch.")
pop_size_string = f"LIMIT {args['pop_size']}" if args['pop_size'] > 0 else ""

min_age_string = str(args['min_age'])
min_dur_string = str(args['min_duration'])
max_dur_string = str(args['max_duration'])
min_day_string = str(float(args['min_duration'])/24)

template_vars = dict(
    limit=pop_size_string, min_age=min_age_string, min_dur=min_dur_string, max_dur=max_dur_string,
    min_day=min_day_string
)

data_df = querier.query(
    query_file=STATICS_QUERY_PATH, extra_template_vars=template_vars
)

end = time.time()
print(f"Took {(end - start)/60} minutes")

Building data from scratch.
Took 0.024374846617380777 minutes


In [9]:
blind_display(data_df, blinded=BLINDED)

data_df.shape:  (103874, 21)


,subject_id,hadm_id,icustay_id,hospital_id,region,icustay_seq,gender,age,ethnicity,los_icu,...,hospital_discharge_offset_min,unit_admit_offset_min,unit_discharge_offset_min,max_hours_unit,max_hours_hospital,discharge_location,mort_hosp,mort_icu,hospital_discharge_year,unit_type


### Add exclusion criteria for remaining queries

In [10]:
querier.add_exclusion_criteria_from_df(data_df, columns=['hadm_id', 'subject_id', 'icustay_id'])

### Extract Labs

In [11]:
start = time.time()
labs_df = querier.query(query_file=LABS_QUERY_PATH)

n_uoms_by_itemid = labs_df.groupby(by=('itemid',))[
    'valueuom_system', 'valueuom_interface'
].nunique().max(axis=1)
n_uoms_by_itemid_dict = n_uoms_by_itemid.to_dict()
set_utoms_by_itemid = labs_df.groupby(by='itemid')[
    ['valueuom_system', 'valueuom_interface']
].agg(set)
# assert n_uoms_by_itemid.max() == 1, {
#     k: v for k, v in n_uoms_by_itemid_dict.items() if v > 1
# }

labs_df.value = pd.to_numeric(labs_df.value, 'coerce')

blind_display(labs_df, blinded=BLINDED)
        
labs_df.drop(inplace=True, columns=['valueuom_system', 'valueuom_interface'])
labs_df = labs_df.groupby(
    ['subject_id', 'hadm_id', 'icustay_id', 'itemid', 'hours_in']
).agg(['mean', 'std', 'count'])

labs_df.columns = labs_df.columns.droplevel(0)
labs_df.columns.names = ['Aggregation Function']

missing_hours_fill = range_unnest(
    data_df.set_index('icustay_id'), 'max_hours_unit', out_col_name='hours_in',
    reset_index=True
)
missing_hours_fill['tmp'] = np.NaN

fill_df = data_df[['icustay_id', 'hadm_id', 'subject_id']].join(
    missing_hours_fill.set_index('icustay_id'), on='icustay_id'
)
N = fill_df.shape[0]

count_per_itemid = labs_df.groupby('itemid')['mean'].count()
measurement_freq_per_itemid = count_per_itemid / N

valid_itemids = measurement_freq_per_itemid[
    measurement_freq_per_itemid > (args['min_labs_thresh']/100)
]

labs_df = labs_df[labs_df.index.get_level_values('itemid').isin(valid_itemids.index)]

labs_df = labs_df.unstack(level=['itemid'])

fill_df.set_index(labs_df.index.names, inplace=True)

labs_df.columns = labs_df.columns.reorder_levels(
    order=['itemid', 'Aggregation Function']
)
labs_df = labs_df.reindex(fill_df.index)

labs_df = labs_df.sort_index(axis=0).sort_index(axis=1)

end = time.time()
print(f"Took {(end - start)/60} minutes")

/crimea/conda_envs/mimic_extract/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: Interpreting tuple 'by' as a list of keys, rather than a single key. Use 'by=[...]' instead of 'by=(...)'. In the future, a tuple will always mean a single key.
  after removing the cwd from sys.path.


labs_df.shape:  (17421186, 8)


,subject_id,hadm_id,icustay_id,hours_in,itemid,value,valueuom_system,valueuom_interface


Took 3.933498477935791 minutes


In [12]:
blind_display(labs_df, blinded=BLINDED)

labs_df.shape:  (7221824, 57)


Empty DataFrame
Columns: [(BUN, count), (BUN, mean), (BUN, std), (Hct, count), (Hct, mean), (Hct, std), (Hgb, count), (Hgb, mean), (Hgb, std), (MCH, count), (MCH, mean), (MCH, std), (MCHC, count), (MCHC, mean), (MCHC, std), (MCV, count), (MCV, mean), (MCV, std), (RBC, count), (RBC, mean), (RBC, std), (RDW, count), (RDW, mean), (RDW, std), (WBC x 1000, count), (WBC x 1000, mean), (WBC x 1000, std), (anion gap, count), (anion gap, mean), (anion gap, std), (bedside glucose, count), (bedside glucose, mean), (bedside glucose, std), (bicarbonate, count), (bicarbonate, mean), (bicarbonate, std), (calcium, count), (calcium, mean), (calcium, std), (chloride, count), (chloride, mean), (chloride, std), (creatinine, count), (creatinine, mean), (creatinine, std), (glucose, count), (glucose, mean), (glucose, std), (platelets x 1000, count), (platelets x 1000, mean), (platelets x 1000, std), (potassium, count), (potassium, mean), (potassium, std), (sodium, count), (sodium, mean), (sodium, std)]
Index: []

[0 rows x 57 columns]

### Extract Vitals

In [13]:
start = time.time()

vitals_df = querier.query(query_file=VITALS_QUERY_PATH)
vitals_df_raw = vitals_df.copy()
blind_display(vitals_df, blinded=BLINDED)

vitals_df.set_index(labs_df.index.names, inplace=True)
for col in vitals_df:
    vitals_df[col] = pd.to_numeric(vitals_df[col], 'coerce')
vitals_df = vitals_df.groupby(vitals_df.index.names).agg(['count', 'mean', 'std'])
    
count_per_vital = vitals_df.loc[:, idx[:, 'mean']].count(axis=0)
measurement_freq_per_vital = count_per_vital / N

invalid_vitals = [x[0] for x in measurement_freq_per_vital[
    measurement_freq_per_vital <= (args['min_labs_thresh']/100)
].index]
vitals_df.drop(columns=[
    (k, a) for k, a in itertools.product(invalid_vitals, ('count', 'mean', 'std'))
], inplace=True)
vitals_df.loc[:, idx[:, 'count']] = vitals_df.loc[:, idx[:, 'count']].fillna(0)
    
vitals_df.columns.names = labs_df.columns.names

labs_and_vitals_df = labs_df.join(vitals_df, how='outer')

end = time.time()
print(f"Took {(end - start) / 60} minutes")

vitals_df.shape:  (14014167, 30)


,subject_id,hadm_id,icustay_id,hours_in,temperature,sao2,heart_rate,respiratory_rate,cvp,etco2,...,noninvasive_systolic,noninvasive_diastolic,noninvasive_mean,paop,cardiac_output,cardiac_input,svr,svri,pvr,pvri


Took 7.261229566733042 minutes


In [14]:
blind_display(labs_and_vitals_df, blinded=BLINDED)

labs_and_vitals_df.shape:  (7237510, 84)


Empty DataFrame
Columns: [(BUN, count), (BUN, mean), (BUN, std), (Hct, count), (Hct, mean), (Hct, std), (Hgb, count), (Hgb, mean), (Hgb, std), (MCH, count), (MCH, mean), (MCH, std), (MCHC, count), (MCHC, mean), (MCHC, std), (MCV, count), (MCV, mean), (MCV, std), (RBC, count), (RBC, mean), (RBC, std), (RDW, count), (RDW, mean), (RDW, std), (WBC x 1000, count), (WBC x 1000, mean), (WBC x 1000, std), (anion gap, count), (anion gap, mean), (anion gap, std), (bedside glucose, count), (bedside glucose, mean), (bedside glucose, std), (bicarbonate, count), (bicarbonate, mean), (bicarbonate, std), (calcium, count), (calcium, mean), (calcium, std), (chloride, count), (chloride, mean), (chloride, std), (creatinine, count), (creatinine, mean), (creatinine, std), (glucose, count), (glucose, mean), (glucose, std), (platelets x 1000, count), (platelets x 1000, mean), (platelets x 1000, std), (potassium, count), (potassium, mean), (potassium, std), (sodium, count), (sodium, mean), (sodium, std), (sao2, count), (sao2, mean), (sao2, std), (heart_rate, count), (heart_rate, mean), (heart_rate, std), (respiratory_rate, count), (respiratory_rate, mean), (respiratory_rate, std), (st1, count), (st1, mean), (st1, std), (st2, count), (st2, mean), (st2, std), (st3, count), (st3, mean), (st3, std), (noninvasive_systolic, count), (noninvasive_systolic, mean), (noninvasive_systolic, std), (noninvasive_diastolic, count), (noninvasive_diastolic, mean), (noninvasive_diastolic, std), (noninvasive_mean, count), (noninvasive_mean, mean), (noninvasive_mean, std)]
Index: []

[0 rows x 84 columns]

### Extract Treatments

In [15]:
def merge_treatment_lists(lists):
    out_list = [set([]) for _ in range(max(len(l) for l in lists.values))]
    for l in lists:
        for i, e in enumerate(l): out_list[i].update([e])
    return out_list

In [16]:
start = time.time()

treatments_df = querier.query(query_file=TREATMENTS_QUERY_PATH)
treatments_df.set_index(labs_and_vitals_df.index.names, inplace=True)
treatments_df['treatment_list'] = treatments_df.treatment_string.str.split('|')
treatments_df = treatments_df.groupby(
    treatments_df.index.names
)[['treatment_list']].agg(merge_treatment_lists)

treatments_df['treatment_area'] = treatments_df['treatment_list'].apply(
    lambda x: x[0] if x else np.NaN
)
treatments_df['specific_treatment'] = treatments_df['treatment_list'].apply(
    lambda x: x[-1] if x else np.NaN
)

end = time.time()
print(f"Took {(end - start)/60} minutes")

Took 0.8994770407676697 minutes


In [17]:
blind_display(treatments_df, blinded=BLINDED)

treatments_df.shape:  (224262, 3)


,,,,treatment_list,treatment_area,specific_treatment
subject_id,hadm_id,icustay_id,hours_in,,,


In [18]:
treatments_for_joining = treatments_df.copy()
treatments_for_joining.columns = pd.MultiIndex.from_tuples([
    ('treatment', c) for c in treatments_for_joining.columns
])

labs_vitals_treatments_df = labs_and_vitals_df.join(
    treatments_for_joining, how='outer'
)

In [19]:
blind_display(labs_vitals_treatments_df, blinded=BLINDED)

labs_vitals_treatments_df.shape:  (7239476, 87)


Empty DataFrame
Columns: [(BUN, count), (BUN, mean), (BUN, std), (Hct, count), (Hct, mean), (Hct, std), (Hgb, count), (Hgb, mean), (Hgb, std), (MCH, count), (MCH, mean), (MCH, std), (MCHC, count), (MCHC, mean), (MCHC, std), (MCV, count), (MCV, mean), (MCV, std), (RBC, count), (RBC, mean), (RBC, std), (RDW, count), (RDW, mean), (RDW, std), (WBC x 1000, count), (WBC x 1000, mean), (WBC x 1000, std), (anion gap, count), (anion gap, mean), (anion gap, std), (bedside glucose, count), (bedside glucose, mean), (bedside glucose, std), (bicarbonate, count), (bicarbonate, mean), (bicarbonate, std), (calcium, count), (calcium, mean), (calcium, std), (chloride, count), (chloride, mean), (chloride, std), (creatinine, count), (creatinine, mean), (creatinine, std), (glucose, count), (glucose, mean), (glucose, std), (platelets x 1000, count), (platelets x 1000, mean), (platelets x 1000, std), (potassium, count), (potassium, mean), (potassium, std), (sodium, count), (sodium, mean), (sodium, std), (sao2, count), (sao2, mean), (sao2, std), (heart_rate, count), (heart_rate, mean), (heart_rate, std), (respiratory_rate, count), (respiratory_rate, mean), (respiratory_rate, std), (st1, count), (st1, mean), (st1, std), (st2, count), (st2, mean), (st2, std), (st3, count), (st3, mean), (st3, std), (noninvasive_systolic, count), (noninvasive_systolic, mean), (noninvasive_systolic, std), (noninvasive_diastolic, count), (noninvasive_diastolic, mean), (noninvasive_diastolic, std), (noninvasive_mean, count), (noninvasive_mean, mean), (noninvasive_mean, std), (treatment, treatment_list), (treatment, treatment_area), (treatment, specific_treatment)]
Index: []

[0 rows x 87 columns]

In [20]:
assert not labs_vitals_treatments_df.isnull().all(axis=0).any()